In [2]:
from huggingface_hub import hf_hub_download
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer, ViTImageProcessor
import torch
import os, shutil
from PIL import Image
import gradio as gr

# # create target model directory
# model_dir = './models/'
# os.makedirs(model_dir, exist_ok=True)

# files_to_download = [
#     "config.json",
#     "flax_model.msgpack",
#     "merges.txt",
#     "special_tokens_map.json",
#     "tokenizer.json",
#     "tokenizer_config.json",
#     "vocab.json",
#     "preprocessor_config.json",
# ]

# "bipin/image-caption-generator" 
model_name = "ipin/image-caption-generator"

# load model
model = VisionEncoderDecoderModel.from_pretrained(model_name)
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# copy files from checkpoint hub:
# for fn in files_to_download:
#     file_path = hf_hub_download("ydshieh/vit-gpt2-coco-en-ckpts", f"ckpt_epoch_3_step_6900/{fn}")
#     shutil.copyfile(file_path, os.path.join(model_dir, fn))

# model = FlaxVisionEncoderDecoderModel.from_pretrained(model_dir)
# feature_extractor = ViTImageProcessor.from_pretrained(model_dir)
# tokenizer = AutoTokenizer.from_pretrained(model_dir)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ModuleNotFoundError: No module named 'jaxlib'

In [3]:
# IMG_PATH = "./data/2.jpeg"
# img_name = IMG_PATH


In [19]:
import numpy as np
prev_preds = []

def predict(img):
    """
    predicts the caption text for the fed image  
    :param img is the imgage feed for generation of caption
    
    :return string, the caption text
    """
    # img = Image.open(img)
    if img.mode != 'RGB':
        img = img.convert(mode="RGB")
    
        
    pixel_values = feature_extractor(images=[img], return_tensors="pt").pixel_values
    # pixel_values = np.array(pixel_values)
    pixel_values = pixel_values.to(device)
    
    max_length = 128
    num_beams = 4

    # get model prediction
    output_ids = model.generate(pixel_values, num_beams=num_beams, max_length=max_length)

    # decode the generated prediction
    preds = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    prev_preds.append(preds)
    # print(prev_preds)
    return preds

In [5]:
predict(img)

NameError: name 'img' is not defined

In [20]:
gr.Interface(fn=predict, inputs=gr.Image(type="pil"), outputs=gr.Textbox("Caption Text")).launch()

Running on local URL:  http://127.0.0.1:7860
Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.
